In [1]:
from huggingface_hub import login
login()

In [2]:
from datasets import load_dataset
dataset = load_dataset('SiberiaSoft/SiberianPersonaChat')

In [66]:
dataset['train'][1]['input']
dataset['train'][10]['input'].split('Собеседник:')[1].split('Ты:')
dataset['train'][10]['input'].split('Собеседник:')[1].split('Ты:')


[' Я пытаюсь уговорить мою бабушку купить собаку. У нее никогда не было домашнего животного, но я думаю, что собака была бы отличным спутником для нее. Как я могу убедить ее?\n',
 ' ']

In [68]:
from tqdm import tqdm

input_messages = []
for data in tqdm(dataset['train']):
    input_messages.append({
        'user': 'instruction',
        'message': data['input'].split('Собеседник')[0]
    })
    if len(data['input'].split('Собеседник:')) > 1 and len(data['input'].split('Собеседник:')[1].split('Ты:')) > 0:
        input_messages.append({
            'user': 'user',
            'message': data['input'].split('Собеседник:')[1].split('Ты:')[0]
        })

    if len(data['input'].split('Собеседник:')) > 2 and len(data['input'].split('Собеседник:')[1].split('Ты:')) > 1 and data['input'].split('Собеседник:')[1].split('Ты:')[1] != ' ':
        input_messages.append({
          'user': 'assistance',
          'message': data['input'].split('Собеседник:')[1].split('Ты:')[1]
        })

    input_messages.append({
        'user': 'assistance',
        'message': data['output']
    })

input_messages[0]

100%|██████████| 448506/448506 [00:18<00:00, 24247.20it/s]


{'user': 'instruction',
 'message': 'Ты парень, певец. Увлекаешься музыкой. Поешь на сцене. Любишь славу и популярность. Продолжи диалог:\n'}

In [87]:
input_messages[30]

{'user': 'user',
 'message': ' Просто ужасно! Не знаю, как я справлюсь с этим экзаменом. Я уже несколько раз не смог сдать его.\n'}

In [84]:
fine_tunning_data = []

start_of_text_token = "<|startoftext|>"
end_of_text_token = "<|endoftext|>"
separator_token = "<|separator|>"

for message in input_messages:
    sender = message['user']
    message = message['message']
    input_text = f"{start_of_text_token}{sender}{separator_token}{message}{end_of_text_token}"
    fine_tunning_data.append(input_text)

fine_tunning_data[0]

'<|startoftext|>instruction<|separator|>Ты парень, певец. Увлекаешься музыкой. Поешь на сцене. Любишь славу и популярность. Продолжи диалог:\n<|endoftext|>'

In [85]:
len(fine_tunning_data)

1576583

In [86]:
import json

with open("./output/finetune_text_corpus_hf.json", 'w', encoding= "utf-8") as f:
     json.dump(fine_tunning_data, f, indent=4, ensure_ascii=False)